In [1]:
import numpy as np
from collections import defaultdict
import copy
import itertools as it

In [2]:
def sums_idxs_dict(sum_vector):
    '''Function that returns a dictionary of the sums values and their indexes in sum vector'''
    return {k:list(np.where(sum_vector == k)[0]) for k in np.unique(sum_vector)}

In [3]:
def axis_sum_idxs_double_dict(arr1, arr2):
    '''
    Function that returns double dictionary of axis, sum value and indexes permutations from
    two arrays. It finds all the possible permutations between the arrays based on every
    value of the vector sum.
    
    Parameters
    ----------
    arr1, arr2: 2D numpy arrays
        Equal dimension arrays to be compared and to extract the permuted indexes.
        
    Returns
    -------
    s_ip: dictionary
        Dictionary of dictionary. First key is the axis. Second keys are sums values, i.e.
        integers. Values are possible permutations (2D numpy arrays) in array notation,
        therefore if the array has only one column, this column represent a new permutation
        for the case of unique sum values, and this permutation should be now permanent.
        
    Example
    -------
    >>> new_a = np.array([[0, 1, 0, 1, 1],
                          [1, 0, 1, 1, 0],
                          [1, 1, 0, 0, 0],
                          [0, 1, 1, 0, 1]])
    >>> new_b = np.array([[1, 1, 0, 0, 1],
                          [0, 1, 1, 1, 0],
                          [0, 0, 0, 1, 1],
                          [1, 0, 1, 1, 0]])
    >>> axis_sum_idxs_double_dict(new_a, new_b)
    {0: {2: array([[0, 2, 3, 4],
             [0, 1, 2, 4]]), 3: array([[1],
             [3]])}, 1: {2: array([[2],
             [2]]), 3: array([[0, 1, 3],
             [0, 1, 3]])}}
    
    To access for example the Cauchy array created by uncertainty in axis 0, this means columns
    uncertainty, and created also by uncertainty in all the columns adding up to 2, we need:
    
    >>> axis_sum_idxs_double_dict(new_a, new_b)[0][2]
    array([[0, 2, 3, 4],
           [0, 1, 2, 4]])
    '''
    
    def sums_idxspermutations_dict(arr1, arr2, ax):
        '''Function that finds all the possible permutations between the arrays based on vector
        sum values.'''
        i = 0
        keys1_list = list(sums_idxs_dict(np.sum(arr1, ax)).keys())
        keys2_list = list(sums_idxs_dict(np.sum(arr2, ax)).keys())
        assert keys1_list == keys2_list
        s_ip = {}
        sidsa1a_values = sums_idxs_dict(np.sum(arr1, ax)).values()
        sidsa2a_values = sums_idxs_dict(np.sum(arr2, ax)).values()
        for v, w in zip(sidsa1a_values, sidsa2a_values):
            assert len(v) == len(w)
            if ax == 0:
                array_value = np.concatenate((np.array([v]), np.array([w])))
            elif ax == 1:
                array_value = np.concatenate((np.array([w]), np.array([v])))
            s_ip[keys1_list[i]] = array_value
            i += 1
        return s_ip
    
    dict_of_dict = {}
    for j in (0, 1):
        dict_of_dict[j] = sums_idxspermutations_dict(arr1, arr2, ax=j)
    return dict_of_dict

In [4]:
def obtain_degeneracies_unique_only(arr1, arr2, ax, unique_sum_value, prints=False):
    '''
    Function that finds the double degeneracy for unique values in two permuted matrices and
    their associated possible permutations.
    
    Parameters
    ----------
    arr1, arr2: 2D numpy arrays
        Equal dimension arrays to be analyzed through their permuted indexes.
    ax: int
        Axis where the unique values vector is present.
    unique_sum_value: int
        Unique sum value to which its associated cauchy column must be used for locating all
        possible permutations for every matrix element type in the perpendicular dimensions.
        
    Returns
    -------
    two_arrays_dict: dict
        Dictionary of two 2D numpy arrays, one for each type of matrix element possible
        permutations. First array for matrix element and assigned key 0, second array for
        matrix element and assigned key 1. They are cauchy arrays for all the possible
        permutations in the perpendicular dimension.
        
    Example
    -------
    >>> new_a = np.array([[0, 1, 0, 1, 1],
                          [1, 0, 1, 1, 0],
                          [1, 1, 0, 0, 0],
                          [0, 1, 1, 0, 1]])
    >>> new_b = np.array([[1, 1, 0, 0, 1],
                          [0, 1, 1, 1, 0],
                          [0, 0, 0, 1, 1],
                          [1, 0, 1, 1, 0]])
                          
    The unique column adding up to 3, with original column index 1 and permuted column index 3,
    won't produce degeneracy in the rows for matrix element 0, which has original row index 1
    and permuted row index 0. But his column will produce a triple degeneracy in the rows for
    matrix element 1:
                          
    >>> obtain_degeneracies_unique_only(new_a, new_b, 0, 3)
    {0: array([[0],
            [1]]), 1: array([[1, 2, 3],
            [0, 2, 3]])}
            
    Similarly, the unique row adding up to 2, will produce the following degeneracies in the
    columns, where it is important to notice the difference in original and permuted
    convention for rows or columns Cauchy arrays:
            
    >>> obtain_degeneracies_unique_only(new_a, new_b, 1, 2)
    {0: array([[2, 3, 4],
            [0, 1, 2]]), 1: array([[0, 1],
            [3, 4]])}
    '''

    dict_of_dicts = axis_sum_idxs_double_dict(arr1, arr2)
    cauchy_column = dict_of_dicts[ax][unique_sum_value]
    if prints:
        print("cauchy_column:")
        print(cauchy_column)
    assert cauchy_column.shape == (2, 1)

    arrs = [arr1, arr2]
    zeros_idxs= []
    ones_idxs= []
    for a in (0, 1):
        if ax == 0:
            unique_vector = arrs[a][:, cauchy_column[a][0]]
        elif ax == 1:
            unique_vector = arrs[a][cauchy_column[1-a][0]]
        zeros_idxs.append(np.where(unique_vector==0)[0])
        ones_idxs.append(np.where(unique_vector==1)[0])
        if prints:
            print("\na", a)
            print("cauchy_column[1-a][0]", cauchy_column[1-a][0])
            print("unique_vector", unique_vector)
            print("np.where(unique_vector==0)[0]", np.where(unique_vector==0)[0])
            print("zeros_idxs", zeros_idxs)
            print("np.where(unique_vector==1)[0]", np.where(unique_vector==1)[0])
            print("ones_idxs", ones_idxs)
    
    two_arrays_dict = {}
    two_arrays_dict[0] = np.concatenate(([zeros_idxs[1-ax]], [zeros_idxs[ax]]))
    two_arrays_dict[1] = np.concatenate(([ones_idxs[1-ax]], [ones_idxs[ax]]))
    return two_arrays_dict

In [5]:
new_a = np.array([[0, 1, 0, 1, 1],
                  [1, 0, 1, 1, 0],
                  [1, 1, 0, 0, 0],
                  [0, 1, 1, 0, 1]])
new_b = np.array([[1, 1, 0, 0, 1],
                  [0, 1, 1, 1, 0],
                  [0, 0, 0, 1, 1],
                  [1, 0, 1, 1, 0]])
obtain_degeneracies_unique_only(new_a, new_b, 1, 2, prints=True)

cauchy_column:
[[2]
 [2]]

a 0
cauchy_column[1-a][0] 2
unique_vector [1 1 0 0 0]
np.where(unique_vector==0)[0] [2 3 4]
zeros_idxs [array([2, 3, 4])]
np.where(unique_vector==1)[0] [0 1]
ones_idxs [array([0, 1])]

a 1
cauchy_column[1-a][0] 2
unique_vector [0 0 0 1 1]
np.where(unique_vector==0)[0] [0 1 2]
zeros_idxs [array([2, 3, 4]), array([0, 1, 2])]
np.where(unique_vector==1)[0] [3 4]
ones_idxs [array([0, 1]), array([3, 4])]


{0: array([[2, 3, 4],
        [0, 1, 2]]), 1: array([[0, 1],
        [3, 4]])}

In [6]:
def pythonic(s):
    '''partition_intersections(x, y) is almost equivalent to list(pythonic(x + y).values()), 
    but lists order is different'''
    all_elements = defaultdict(list)
    for i, ss in enumerate(s):
        for elem in ss:
            all_elements[elem].append(i)
    reversed = defaultdict(list)
    for k, v in all_elements.items():
        reversed[frozenset(v)].append(k)
    return reversed#.keys()#.values()#list(reversed.values())

In [7]:
def partition_intersections(x, y):
    '''
    Function that finds all Cartesian product intersections between two partitions x and y
    of the same set. For example, partition_intersections(x, y) returns roughly the same as:
    [np.intersect1d(a, b) for a in x for b in y]
    
    Parameters
    ----------
    x, y: lists
        Lists of lists, or list of 1D numpy arrays. Flattened lists of x and y should be equal.
    
    Returns
    -------
    all_lists: list
        List of lists with all Cartesian product intersections between x and y.
    
    Example
    -------
    >>> u = [[1, 2], [3, 4, 5], [6, 7, 8, 9, 10]]
    >>> v = [[1, 3, 6, 7], [2, 4, 5, 8, 9, 10]]
    >>> partition_intersections(u, v)
    [[1], [2], [3], [4, 5], [6, 7], [8, 9, 10]]
    '''
    flattened_x = np.sort(np.concatenate((x), axis=None))
    flattened_y = np.sort(np.concatenate((y), axis=None))
    assert (flattened_x == flattened_y).all()
    all_s = []
    for sx in x:
        for sy in y:
            ss = list(filter(lambda i:i in sx, sy))
            if ss:
                all_s.append(ss)#np.array(ss))
    return all_s
    '''all_lists = list(pythonic(x + y).values()) #this is almost equivalent, but lists order is different
    return all_lists'''

In [8]:
def fullaxis_uniqueonly_degeneracies_intersection(fullaxis_dict, uniqueonly_dict):
    '''
    Function that applies partition_intersections among rows or cauchy arrays.
    
    Parameters
    ----------
    fullaxis_dict: dict
        Dictionary of all sums and their indexes degeneracy for a given axis, obtained from
        axis_sum_idxs_double_dict.
    uniqueonly_dict: dict
        Dictionary of both degeneracies for given unique sum in axis perpendicular to the
        previous one, obtained from obtain_degeneracies_unique_only.
    
    Returns
    -------
    final_list: list
        List of Cauchy arrays with partially complete degeneracy for given axis.
    
    Example
    -------
    >>> new_a = np.array([[0, 1, 0, 1, 1],
                          [1, 0, 1, 1, 0],
                          [1, 1, 0, 0, 0],
                          [0, 1, 1, 0, 1]])
    >>> new_b = np.array([[1, 1, 0, 0, 1],
                          [0, 1, 1, 1, 0],
                          [0, 0, 0, 1, 1],
                          [1, 0, 1, 1, 0]])
                          
    If we want to find intersection of degeneracies due to all columns (axis 0), knowing the
    degeneracies obtained due to rows (axis 1) with unique sum value 2, the respective Cauchy
    arrays for columns are:
    
    >>> asiddnanb0 = axis_sum_idxs_double_dict(new_a, new_b)[0]
    >>> oduonanb12 = obtain_degeneracies_unique_only(new_a, new_b, 1, 2)
    >>> fullaxis_uniqueonly_degeneracies_intersection(asiddnanb0, oduonanb12)
    [array([[2, 3, 4],
            [0, 1, 2]]), array([[0],
            [4]]), array([[1],
            [3]])]
    '''
    
    def dic_row(dic, row):
        '''Function that contructs list of 1D numpy arrays (rows) from row of dictionary of
        2D numpy arrays'''
        return [arr[row] for arr in dic.values()]
    
    pdp0 = partition_intersections(dic_row(fullaxis_dict, 0), dic_row(uniqueonly_dict, 0))
    pdp1 = partition_intersections(dic_row(fullaxis_dict, 1), dic_row(uniqueonly_dict, 1))
    #print(pdp0)
    #print(pdp1)
    final_list = []
    for m, n in zip(pdp0, pdp1):
        final_list.append(np.concatenate(([m], [n])))
    return final_list

In [9]:
new_a = np.array([[0, 1, 0, 1, 1],
                  [1, 0, 1, 1, 0],
                  [1, 1, 0, 0, 0],
                  [0, 1, 1, 0, 1]])
new_b = np.array([[1, 1, 0, 0, 1],
                  [0, 1, 1, 1, 0],
                  [0, 0, 0, 1, 1],
                  [1, 0, 1, 1, 0]])
asiddnanb0 = axis_sum_idxs_double_dict(new_a, new_b)[0]
oduonanb12 = obtain_degeneracies_unique_only(new_a, new_b, 1, 2)
fullaxis_uniqueonly_degeneracies_intersection(asiddnanb0, oduonanb12)

[array([[2, 3, 4],
        [0, 1, 2]]), array([[0],
        [4]]), array([[1],
        [3]])]

In [10]:
def unique_sum_values(sum_vector):
    '''Function that returns the unique sum values for a given (row or column) sum vector'''
    sid = sums_idxs_dict(sum_vector)
    return [k for k, v in sid.items() if len(v)==1]

In [11]:
def array_to_latex(arr, square_brackets=True):
    c_s = 'r'
    if square_brackets:
        print("\\left\{\\left(\\begin{array}{" + c_s*(arr.shape[1]) + "}")
    else:
        print("\\left(\\begin{array}{" + c_s*(arr.shape[1]) + "}")
    for r, row in enumerate(arr):
        elel = ''
        for e, el in enumerate(np.char.mod('%d', row)):
            if e != arr.shape[1]-1:
                elel += el + ' & '
            else:
                elel += el + ' '
        if r != arr.shape[0]-1:
            elel += '\\\\'
        print(elel)
    if square_brackets:
        print("\\end{array}\\right)\\right\}")
    else:
        print("\\end{array}\\right)")
array_to_latex(np.array([[6, 5], [4, 6]]))#, square_brackets=False)

\left\{\left(\begin{array}{rr}
6 & 5 \\
4 & 6 
\end{array}\right)\right\}


In [12]:
def cauchy_to_latex(arr):
    print("\\begin{equation}")
    array_to_latex(arr, square_brackets=False)
    print("\end{equation}")
cauchy_to_latex(np.array([[6, 5], [4, 6]]))

\begin{equation}
\left(\begin{array}{rr}
6 & 5 \\
4 & 6 
\end{array}\right)
\end{equation}


In [13]:
def dicti_to_latex(dicti, index_alignment='left'):
    print("\\begin{align}")
    j = 0
    for k, v in dicti.items():
        if index_alignment=='left':
            print(k, '&:')
        if v.shape == (2, 1):
            array_to_latex(v, square_brackets=False)
        else:
            array_to_latex(v)
        if index_alignment=='right':
            print('&:', k)
        if j != len(dicti)-1:
            print('\\\\')
        j += 1
    print("\end{align}")

In [14]:
def horizontal_arrays_latex(arrays_list, only_Cauchy=False):
    print("\\begin{equation}")
    for i, ar in enumerate(arrays_list):
        if only_Cauchy:
            array_to_latex(ar+1, square_brackets=False)
        else:
            if ar.shape == (2, 1):
                array_to_latex(ar+1, square_brackets=False)
            else:
                array_to_latex(ar+1)
        if i != len(arrays_list)-1:
            print(",\\")
    print("\end{equation}")

In [15]:
def FINAL(a, b, prints=False, latex_prints=False):
    #print("***initial FINAL***")
    partial_permutations_dict = {}
    for j in (1, 0):
        us_a = unique_sum_values(np.sum(a, axis=j))
        us_b = unique_sum_values(np.sum(b, axis=j))
        assert us_a == us_b
        
        if us_a:
            if latex_prints:
                unique_a_idxs = [np.where(np.sum(a, axis=j)==k)[0][0]+1 for k in us_a]
                unique_b_idxs = [np.where(np.sum(b, axis=j)==k)[0][0]+1 for k in us_b]
                print("")
                if j == 1:
                    print("The unique rows sum values $", us_a, "$ permutations are represented in Cauchy notation as")
                    cauchy_to_latex(np.concatenate(([unique_b_idxs], [unique_a_idxs])))
                elif j == 0:
                    print("The unique columns sum values $", us_a, "$ permutations are represented in Cauchy notation as")
                    cauchy_to_latex(np.concatenate(([unique_a_idxs], [unique_b_idxs])))
            my_dict = {}
            c = 0
            p_p = []
            asiddabj = axis_sum_idxs_double_dict(a, b)[1-j]
            if latex_prints:
                if j == 1:
                    print("In the perpendicular dimension of the columns, the degeneracy is represented by the following permutations with the respective sum indexes coming next")
                elif j == 0:
                    print("In the perpendicular dimension of the rows, the degeneracy is represented by the following permutations with the respective sum indexes coming next")
                #print(asiddabj)
                asiddabj_plusone = {k:v+1 for k, v in asiddabj.items()}
                #print(asiddabj_plusone)
                dicti_to_latex(asiddabj_plusone, index_alignment='right')
                print("\\begin{enumerate}")

            ### To replace FOR loop for a WHILE loop, updating the degeneracies:
            for u in us_a:
                #asiddabj = axis_sum_idxs_double_dict(a, b)[1-j]
                oduoabju = obtain_degeneracies_unique_only(a, b, j, u)
                fa_uo_di = fullaxis_uniqueonly_degeneracies_intersection(asiddabj, oduoabju)
                if latex_prints:
                    if j == 1:
                        print("\item For the unique rom sum value {}, the perpendicular degeneracies of its zeros and ones is".format(u))#, oduoabju)
                    if j == 0:
                        print("\item For the unique column sum value {}, the perpendicular degeneracies of its zeros and ones is".format(u))
                    new_oduoabju = {k: v for k, v in oduoabju.items() if v.size!=0}
                    new_oduoabju_plusone = {k:v+1 for k, v in new_oduoabju.items()}
                    dicti_to_latex(new_oduoabju_plusone, index_alignment='left')
                    if j == 1:
                        print("thus the intersection of it with its perpendicular full columns degeneracy is")
                    if j == 0:
                        print("thus the intersection of it with its perpendicular full rows degeneracy is")
                    horizontal_arrays_latex(fa_uo_di)
                my_dict[u] = fa_uo_di
                c += 1
                if c == 1:
                    p_p = my_dict[us_a[c-1]]
                if c > 1:
                    t_dict = {}
                    for i in (0, 1):
                        rows_minus1 = [sublist[i] for sublist in my_dict[us_a[c-1]]]
                        rows_partial = [sublist[i] for sublist in p_p]
                        t_dict[i] = partition_intersections(rows_minus1, rows_partial)
                    p_p = [np.concatenate(([m], [n])) for m, n in zip(t_dict[0], t_dict[1])]
            ### To replace FOR loop for a WHILE loop, updating the degeneracies

            if latex_prints:
                print("\end{enumerate}")
            if p_p:
                partial_permutations_dict[1-j] = p_p
            else:
                asiddabj = axis_sum_idxs_double_dict(a, b)[1-j]
                partial_permutations_dict[1-j] = [v for v in asiddabj.values()]
    
    if latex_prints:
        print("")
        if 0 in partial_permutations_dict.keys():
            print("Finally, all the obtained partial permutations for columns are")#, partial_permutations_dict)
            horizontal_arrays_latex(partial_permutations_dict[0])
        if 1 in partial_permutations_dict.keys():
            print("and all the obtained partial permutations for rows are")#, partial_permutations_dict)
            horizontal_arrays_latex(partial_permutations_dict[1])

    #print("***final FINAL***")
    return partial_permutations_dict

In [16]:
def ready_nonready_arrays(new_a, new_b, ab_dict, prints=False, prints_counter=False):
    ready_arrays_dict = {}
    nonready_arrays_dict = {}
    for k, v in ab_dict.items():
        nonready_arrays = []
        if prints:
            print("\nk", k)
            print("v", v)
        r = n = 0
        for w in v:
            if w.shape[1] == 1:
                r += 1
                if prints:
                    print(r, "READY:")
                    print(w)
                if r == 1:
                    ready_array = w
                    if prints:
                        print("ready_array:")
                        print(ready_array)
                else:
                    ready_array = np.concatenate((ready_array, w), axis=1)
                    if prints:
                        print("ready_array:")
                        print(ready_array)
            else:
                n += 1
                if prints:
                    print(n, "not ready:")
                    print(w)
                    print("nonready_arrays", nonready_arrays)
                nonready_arrays.append(w)
        if prints_counter:
            print("Final counters (r: ready, n: not ready)")
            print("r:", r)
            print("n:", n)
        if r != 0:
            ready_arrays_dict[k] = ready_array
        if n != 0:
            nonready_arrays_dict[k] = nonready_arrays
    return ready_arrays_dict, nonready_arrays_dict

In [17]:
def permuted_order_cauchy_to_python(arr, ax):
    '''
    Example
    -------
    >>> cauchy = np.array([[1, 0, 2, 3, 5],
                           [0, 2, 3, 1, 5]]) 
    >>> poctpCOL = permuted_order_cauchy_to_python(cauchy, 1)
    >>> poctpCOL    
    array([2, 0, 3, 1, 5])
    >>> poctpROW = permuted_order_cauchy_to_python(cauchy, 0)
    >>> poctpROW
    array([1, 3, 0, 2, 5])
    '''
    cauchy_order = arr[:, np.argsort(arr[1-ax])]
    python_permutation_order = cauchy_order[ax]    
    return python_permutation_order

In [18]:
def partial_permuted_order_cauchy_to_python(arr, ax, length):
    for el in range(length):
        #print("el", el)
        if el not in arr[0]:##ax]:
            if el not in arr[1]:##-ax]:
                ap = np.array([[el], [el]])
                arr = np.concatenate((arr, ap), axis=-1)
                #print(arr)
            else:
                #print("np.argwhere(arr==el)", np.argwhere(arr==el))
                assert np.argwhere(arr==el)[0][0]==1##(1-ax)
                e = el
                i = 0##ax
                #print("init i", i)
                while e in arr[1-i%2]:
                    #print(" i", i)
                    #print(" 1-i%2", 1-i%2)
                    #print(" e", e)
                    row_idx = 1-i%2
                    #print(" row_idx", row_idx)
                    #print(" np.argwhere(arr==e):")
                    #print(np.argwhere(arr==e))
                    new_arg = np.argwhere(arr==e)
                    #print(" new_arg[:,0] == row_idx", new_arg[:,0] == row_idx)
                    #print(" np.argwhere(new_arg[:,0]==row_idx)", np.argwhere(new_arg[:,0]==row_idx))
                    chosen_row = np.argwhere(new_arg[:,0]==row_idx)[0]
                    #print(" chosen_row", chosen_row)
                    col_idx = np.argwhere(arr==e)[chosen_row[0]][1]
                    #print(" col_idx", col_idx)
                    e = arr[0, col_idx]##ax, col_idx]
                    #print(" new e", e)
                    i += 1
                #print("final e", e)
                '''if ax==0:
                    ap = np.array([[el], [e]])
                elif ax==1:
                    ap = np.array([[e], [el]])'''
                ap = np.array([[el], [e]])
                arr = np.concatenate((arr, ap), axis=-1)
                #print(arr)
    #print("FINAL arr")
    #print(arr)
    return permuted_order_cauchy_to_python(arr, ax)

In [19]:
def arrays_and_more_to_latex(arr, *idxs_to_bold, square_bracket=False):
    big_arr = np.concatenate((np.sum(arr, axis=1).reshape((arr.shape[0],1)), arr), axis=1)
    c_s = 'c'
    if square_bracket:
        print("[$\\begin{blockarray}{" + c_s*(big_arr.shape[1]) + "}")
    else:
        print("$\\begin{blockarray}{" + c_s*(big_arr.shape[1]) + "}")
    print("\\begin{block}{c[" + c_s*(big_arr.shape[1]-1) + "]}")
    for r, row in enumerate(big_arr):
        elel = ''
        for e, el in enumerate(np.char.mod('%d', row)):
            if idxs_to_bold:
                #print("idxs_to_bold[0]", idxs_to_bold[0])
                if (r, e) in idxs_to_bold[0]: #and idxs_to_bold not None:
                    el = '\\mathbf{' + el + '}'
            if e != big_arr.shape[1]-1:
                elel += el + ' & '
            else:
                elel += el + ' \\'
        if r == 0:
            elel += 'topstrut \\\\'
        elif r == big_arr.shape[0]-1:
            elel += 'botstrut \\\\'
        else:
            elel += '\\'
        print(elel)
    print("\\end{block}")
    els = ''
    for el in np.char.mod('%d', np.sum(arr, axis=0)):
        els += ' & ' + el
    print(els)
    if square_bracket:
        print("\end{blockarray}$]")
    else:
        print("\end{blockarray}$")

In [20]:
def python_to_forest_of_arrays(arr1, arr2, *tup_nr_to_bold, s_b=True, ready_dict_for_partial_array=None):
    if tup_nr_to_bold:
        idxs_1_to_bold = [(x, y+1) for x in tup_nr_to_bold[0][1][1] for y in tup_nr_to_bold[0][0][0]] #y+1 because there is the sum column at the beginning
        idxs_2_to_bold = [(x, y+1) for x in tup_nr_to_bold[0][1][0] for y in tup_nr_to_bold[0][0][1]] #y+1 because there is the sum column at the beginning
    print("\\newline")
    print("\\begin{forest}")
    if tup_nr_to_bold:
        arrays_and_more_to_latex(arr1, idxs_1_to_bold, square_bracket=s_b)
        print("[$\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;$]")
        if ready_dict_for_partial_array is not None:
            partial_cols = ready_dict_for_partial_array[0]
            partial_rows = ready_dict_for_partial_array[1]
            partial_cols_permutation = partial_permuted_order_cauchy_to_python(partial_cols, 0, arr1.shape[1])
            partial_rows_permutation = partial_permuted_order_cauchy_to_python(partial_rows, 1, arr1.shape[0])
            arr12 = arr1[partial_rows_permutation][:, partial_cols_permutation]
            idxs_12_to_bold = [(list(partial_rows_permutation).index(i[0]), list(partial_cols_permutation).index(i[1]-1)+1) for i in idxs_1_to_bold] #-1 & +1 because there is the sum column at the beginning
            arrays_and_more_to_latex(arr12, idxs_12_to_bold, square_bracket=s_b)#, square_bracket=s_b)#
            print("[$\;\;\;\;$]")
        arrays_and_more_to_latex(arr2, idxs_2_to_bold, square_bracket=s_b)
    else:
        arrays_and_more_to_latex(arr1, square_bracket=s_b)
        print("[$\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;$]")
        arrays_and_more_to_latex(arr2, square_bracket=s_b)
    print("\end{forest}")
    print("")#\\newline")

In [21]:
def all_indexes_permutations(arr):
    '''
    Example
    -------
    >>> all_indexes_permutations(np.reshape(np.arange(6), (2,3)))
    array([[[0, 1, 2],
            [3, 4, 5]],

           [[0, 1, 2],
            [3, 5, 4]],

           [[0, 1, 2],
            [4, 3, 5]],

           [[0, 1, 2],
            [4, 5, 3]],

           [[0, 1, 2],
            [5, 3, 4]],

           [[0, 1, 2],
            [5, 4, 3]]])
    '''
    assert arr.shape[0] == 2
    cases_number = np.math.factorial(arr.shape[1])
    multi_array = np.zeros((cases_number, arr.shape[0], arr.shape[1]), dtype=int)
    j = 0
    for i in it.permutations(arr[1]):
        new_array = np.stack((arr[0], i))
        multi_array[j] = new_array
        j += 1
    return multi_array

# ITERATIVE FUNCTION:

In [22]:
#def update_dicts(to_update_ready_dict, to_update_nonready_dict, AA, BB, idx0=0, idx1=0, prints=False):
    #tup_nr = (to_update_nonready_dict[0][idx0], to_update_nonready_dict[1][idx1])
def update_dicts(to_update_ready_dict, to_update_nonready_dict, AA, BB, tup_nr, u_t=False, prints=False, latex_prints=False):
    A = AA[tup_nr[1][1]][:, tup_nr[0][0]]
    B = BB[tup_nr[1][0]][:, tup_nr[0][1]]
    if latex_prints:
        print("For the degeneracy pair of columns and rows")
        horizontal_arrays_latex(tup_nr)
        print("from")
        python_to_forest_of_arrays(AA, BB, tup_nr)
        print("we obtain the submatrices")
        python_to_forest_of_arrays(A, B)
    if prints:
        print("\n++++++++++++INI:update_dicts++++++++++++")
        print("A:")
        print(A)
        print("B:")
        print(B)
    r_n_tuple = ready_nonready_arrays(A, B, FINAL(A, B, prints=prints, latex_prints=latex_prints))
    ready_AB_dict = r_n_tuple[0]
    nonready_AB_dict = r_n_tuple[1]
    ready_sub_dict = {k: tup_nr[k][np.arange(2)[:, None], ready_AB_dict[k]] for k in 
                      ready_AB_dict.keys()}

    if not ready_sub_dict:
        #print("There are not new found permutations during this iteration")
        u_t = True
        return to_update_ready_dict, to_update_nonready_dict, u_t

    nonready_sub_dict = {k: [tup_nr[k][np.arange(2)[:, None], v] for v in 
                             nonready_AB_dict[k]] for k in nonready_AB_dict.keys()}
    if prints:
        print("ready_sub_dict", ready_sub_dict)
        print("nonready_sub_dict", nonready_sub_dict)
    for ax in ready_sub_dict:
        if prints:
            print("AXIS ax", ax, "analysis for ready_sub_dict:")
            print("initial to_update_ready_dict[ax]", to_update_ready_dict[ax])
            print("ready_sub_dict[ax]", ready_sub_dict[ax])
        for col in ready_sub_dict[ax].T:
            cauchy_col = np.reshape(col, (2, 1))
            if not (to_update_ready_dict[ax] == cauchy_col).all(axis=0).any():
                to_update_ready_dict[ax] = np.concatenate((to_update_ready_dict[ax],
                                                           cauchy_col), axis=1)
                if prints:
                    print("to_update_ready_dict[ax]:")
                    print(to_update_ready_dict[ax])
                    print("initial to_update_nonready_dict[ax]", to_update_nonready_dict[ax])
                nr0_bool = [cauchy_col[0] in arr[0] for arr in to_update_nonready_dict[ax]]
                nr1_bool = [cauchy_col[1] in arr[1] for arr in to_update_nonready_dict[ax]]
                assert nr0_bool.index(True) == nr1_bool.index(True)
                true_arr_idx = nr0_bool.index(True)
                arr_to_update = to_update_nonready_dict[ax][true_arr_idx]
                #print("initial arr_to_update", arr_to_update)
                idxs = [(i, np.where(arr_to_update[i]==cauchy_col[i])[0][0]) for i in (0, 1)]
                #print("idxs", idxs)
                atus0 = arr_to_update.shape[0]
                atus1 = arr_to_update.shape[1]
                idxs = [i*atus1+j for i, j in idxs]
                #print("idxs", idxs)
                arr_to_update = np.delete(arr_to_update, idxs).reshape(atus0, atus1-1)
                #print("final arr_to_update", arr_to_update)
                if arr_to_update.shape == (2, 1):
                    to_update_ready_dict[ax] = np.concatenate((to_update_ready_dict[ax],
                                                               arr_to_update), axis=1)
                    to_update_nonready_dict[ax].pop(true_arr_idx)
                    if prints:
                        print("if, to_update_ready_dict instead:")
                        print("to_update_ready_dict[ax]", to_update_ready_dict[ax])
                        print("to_update_nonready_dict[ax]", to_update_nonready_dict[ax])
                else:
                    to_update_nonready_dict[ax][true_arr_idx] = arr_to_update
                    if prints:
                        print("else, to_update_nonready_dict[ax] now:")
                        print("to_update_nonready_dict[ax]", to_update_nonready_dict[ax])
    for ax in nonready_sub_dict:
        if prints:
            print("AXIS ax", ax, "analysis for nonready_sub_dict:")
            print("initial to_update_nonready_dict[ax]", to_update_nonready_dict[ax])
            print("nonready_sub_dict[ax]", nonready_sub_dict[ax])
        nrdax0 = [a[0] for a in to_update_nonready_dict[ax]]
        nrsdax0 = [a[0] for a in nonready_sub_dict[ax]]
        nrdax1 = [a[1] for a in to_update_nonready_dict[ax]]
        nrsdax1 = [a[1] for a in nonready_sub_dict[ax]]
        PDP0 = [v for v in pythonic(nrdax0 + nrsdax0).values()]
        PDP1 = [pythonic(nrdax1 + nrsdax1)[k] for k in pythonic(nrdax0 + nrsdax0).keys()]
        new_list = []
        for m, n in zip(PDP0, PDP1):
            new_list.append(np.concatenate(([m], [n])))
        idx_to_pop = []
        for c, arr in enumerate(new_list):
            if ((arr.shape == (2, 1)) and
                not (to_update_ready_dict[ax] == arr).all(axis=0).any()):
                to_update_ready_dict[ax] = np.concatenate((uto_update_ready_dict[ax], arr),
                                                          axis=1)
                if prints:
                    print("To update in to_update_ready_dict[ax] the arr:", arr)
                    print("to_update_ready_dict[ax]:")
                    print(to_update_ready_dict[ax])
                idx_to_pop.append(c)
            elif (arr.shape == (2, 1)) and (to_update_ready_dict[ax] == arr).all(axis=0).any():
                if prints:
                    print("Already updated in to_update_ready_dict[ax] the arr:", arr)
                idx_to_pop.append(c)
        for i, idx in enumerate(idx_to_pop):
            new_list.pop(idx - i) #because when poping an index, it changes the future indexes
        to_update_nonready_dict[ax] = new_list
        if prints:
            print("to_update_nonready_dict[ax]", to_update_nonready_dict[ax])

    if prints:
        print("++++++++++++END:update_dicts++++++++++++\n")
    return to_update_ready_dict, to_update_nonready_dict, u_t

In [23]:
def blablabla(A, B, prints=False, latex_prints=False):
    #print("*********INI:blablabla*********")
    python_to_forest_of_arrays(A, B)
    ready_nonready_tuple = ready_nonready_arrays(A, B, FINAL(A, B, prints=prints, latex_prints=latex_prints))
    ready_dict = ready_nonready_tuple[0]
    nonready_dict = ready_nonready_tuple[1]
    if prints:
        print("\n")
        print("*********INI:blablabla*********")
        print("ready_dict", ready_dict)
        print("nonready_dict", nonready_dict)
    checked_tuples = []
    SsS = 0
    pairs = [-1]
    while (0 in nonready_dict and 1 in nonready_dict) and (SsS < len(pairs)): #if 0 and/or 1 are not in nonready_dict: DON'T FORGET TO INCLUDE LATER!
        pairs = []
        for pair in it.product(range(len(nonready_dict[0])), range(len(nonready_dict[1]))):
            pairs.append(pair)
        #if latex_prints:
        #    print("*********1st:while*********")
        #    print("pairs", pairs)
        unchanged_tuple = True
        SsS = 0
        while unchanged_tuple and (SsS < len(pairs)):
            if latex_prints:
                #print("*****2nd:while*****")
                #print("SsS", SsS)
                #print("pairs[SsS]", pairs[SsS])
                #print("ready_dict", ready_dict)
                #print("nonready_dict", nonready_dict)
                #print("")
                print("We can visualize this partial permutations, where the actual partially permuted matrix is in the middle, highlightning the elements where the permutations are still to be found as")
                all_tup_nr = (np.concatenate(nonready_dict[0], axis=-1), np.concatenate(nonready_dict[1], axis=-1))
                python_to_forest_of_arrays(A, B, all_tup_nr, s_b=True, ready_dict_for_partial_array=ready_dict)
                print("")
                print("An alternative way to understand this results is to consider separately the permutations that are ready, for columns and rows respectively")
                horizontal_arrays_latex([v for v in ready_dict.values()], only_Cauchy=True)
                print("where of course we condensed all the ready permutations into a single Cauchy array. Especially important are the permutations that are not ready, first for columns")
                horizontal_arrays_latex(nonready_dict[0])
                print("and then for rows")
                horizontal_arrays_latex(nonready_dict[1])
                if checked_tuples:
                    print("So far, the following tuples have been reviewed")
                    for tup in checked_tuples:
                        horizontal_arrays_latex(tup, only_Cauchy=True)
            #ready_dict, nonready_dict, br = update_dicts(ready_dict, nonready_dict, A, B,
            #                                             prints=prints)
            i0 = pairs[SsS][0]#0
            i1 = pairs[SsS][1]#0
            tup_nr = (nonready_dict[0][i0], nonready_dict[1][i1]) #optional, only to print #:NOT ANYMORE!
            #if latex_prints:
            #    print("In particular, the following tuple is analyzed", tup_nr)
            if any(np.array_equal(tup_nr[0], t[0]) and np.array_equal(tup_nr[1], t[1]) for t in checked_tuples):
                if latex_prints:
                    print("This tuple was already analyzed, therefore we skip it")
            else:
                ready_dict, nonready_dict, unchanged_tuple = update_dicts(ready_dict, nonready_dict, A, B, tup_nr, prints=prints, latex_prints=latex_prints)
                checked_tuples.append(tup_nr)
                if latex_prints:
                    print("This tuple has been just appended to the checked tuples, as usual, with indexes for columns and rows")
                    horizontal_arrays_latex(tup_nr)#, only_Cauchy=True)
            SsS += 1

            if latex_prints:
                print("The", SsS, "th iteration here results in the ready permutations")
                horizontal_arrays_latex([v for v in ready_dict.values()], only_Cauchy=True)
                print("The non ready permutations for columns are")
                horizontal_arrays_latex(nonready_dict[0])
                print("and the non ready permutations for rows are")
                horizontal_arrays_latex(nonready_dict[1])

    #print("*********END:blablabla*********")
    return ready_dict, nonready_dict

## Testing
## Input dimensions of random matrix to create (rows, cols): dim
## Input maximum number of brute force permutations to try: max_perm

### The (random) input array a and its random permutation b (if you get an error, restart from here):

In [24]:
#9x6
a = np.array([[1, 1, 1, 0, 1, 0],
              [0, 1, 1, 0, 1, 1],
              [1, 1, 0, 0, 1, 0],
              [0, 1, 0, 0, 1, 1],
              [1, 0, 1, 1, 0, 1],
              [0, 1, 0, 1, 1, 1],
              [0, 1, 0, 1, 1, 0],
              [0, 0, 1, 1, 0, 0],
              [0, 1, 1, 1, 1, 1]])
b = np.array([[1, 0, 0, 1, 1, 1],
              [1, 0, 0, 0, 1, 1],
              [1, 0, 1, 1, 1, 1],
              [1, 0, 1, 0, 1, 1],
              [0, 1, 0, 0, 1, 1],
              [0, 0, 1, 1, 0, 0],
              [0, 1, 0, 1, 1, 1],
              [0, 0, 1, 0, 1, 1],
              [1, 1, 1, 1, 0, 0]])

In [25]:
'''dim = (9, 6)
a = np.random.randint(2, size=dim)
ar = np.arange(a.shape[0])
np.random.shuffle(ar)
ac = np.arange(a.shape[1])
np.random.shuffle(ac)
b = a[ar][:,ac]'''

'dim = (9, 6)\na = np.random.randint(2, size=dim)\nar = np.arange(a.shape[0])\nnp.random.shuffle(ar)\nac = np.arange(a.shape[1])\nnp.random.shuffle(ac)\nb = a[ar][:,ac]'

#### NOW we execute the full algorithm:

In [26]:
urd, unrd = blablabla(a, b, prints=False, latex_prints=True)

\newline
\begin{forest}
[$\begin{blockarray}{ccccccc}
\begin{block}{c[cccccc]}
4 & 1 & 1 & 1 & 0 & 1 & 0 \topstrut \\
4 & 0 & 1 & 1 & 0 & 1 & 1 \\
3 & 1 & 1 & 0 & 0 & 1 & 0 \\
3 & 0 & 1 & 0 & 0 & 1 & 1 \\
4 & 1 & 0 & 1 & 1 & 0 & 1 \\
4 & 0 & 1 & 0 & 1 & 1 & 1 \\
3 & 0 & 1 & 0 & 1 & 1 & 0 \\
2 & 0 & 0 & 1 & 1 & 0 & 0 \\
5 & 0 & 1 & 1 & 1 & 1 & 1 \botstrut \\
\end{block}
 & 3 & 7 & 5 & 5 & 7 & 5
\end{blockarray}$]
[$\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;$]
[$\begin{blockarray}{ccccccc}
\begin{block}{c[cccccc]}
4 & 1 & 0 & 0 & 1 & 1 & 1 \topstrut \\
3 & 1 & 0 & 0 & 0 & 1 & 1 \\
5 & 1 & 0 & 1 & 1 & 1 & 1 \\
4 & 1 & 0 & 1 & 0 & 1 & 1 \\
3 & 0 & 1 & 0 & 0 & 1 & 1 \\
2 & 0 & 0 & 1 & 1 & 0 & 0 \\
4 & 0 & 1 & 0 & 1 & 1 & 1 \\
3 & 0 & 0 & 1 & 0 & 1 & 1 \\
4 & 1 & 1 & 1 & 1 & 0 & 0 \botstrut \\
\end{block}
 & 5 & 3 & 5 & 5 & 7 & 7
\end{blockarray}$]
\end{forest}


The unique rows sum values $ [2, 5] $ permutations are represented in Cauchy notation as
\begin{equation}
\left(\begin{array

##### For the row and colum indexes where we still have a degeneracy, a brute force approach is implemented, where every row in the cauchy array contributes with the factorial of its number of columns to the total, and it is calculated beforehand in brute_force_permutations

In [27]:
urd

{0: array([[0, 5, 2, 3],
        [1, 0, 3, 2]]), 1: array([[5, 4, 2, 1, 7],
        [7, 2, 8, 3, 6]])}

In [28]:
unrd[0]

[array([[1, 4],
        [4, 5]])]

In [29]:
unrd[1]

[array([[0, 3],
        [1, 5]]), array([[6, 8],
        [0, 4]])]

Finally, when implementing the brute force permutations of the remaining degenerate indexes, the (unique) counter index where the permutation between a and b is found is called successful_permutation_counter

In [30]:
def final_brute_force_latex(ready, nonready, max_perm, latex_prints=False):
    col_dims = [arr.shape[1] for arr in [a for sublist in unrd.values() for a in sublist]]
    brute_force_permutations = np.prod(col_dims)
    if latex_prints:
        print("")
        print("Now it comes the final part of the process. Currently, the ready permutations are")
        horizontal_arrays_latex([v for v in ready.values()], only_Cauchy=True)
        print("The non ready permutations for columns are")
        horizontal_arrays_latex(nonready[0])
        print("and the non ready permutations for rows are")
        horizontal_arrays_latex(nonready[1])
        print("For the row and column indexes where we still have a degeneracy, a brute force approach is implemented, where every row in the Cauchy array contributes with the factorial of its number of columns to the total, and all the possible permutations are tested. Looking at the dimensions of the nonready Cauchy arrays $", tuple(col_dims), "$ we find a total of", brute_force_permutations, "brute force permutations to be tested.")

    if brute_force_permutations < max_perm:
        aip0 = [all_indexes_permutations(arr) for arr in unrd[0]]
        aip1 = [all_indexes_permutations(arr) for arr in unrd[1]]
        c = 0
        for j in it.product(*aip0):
            perm0 = np.concatenate(j, axis=-1)
            #print(perm0)
            funrd = {}
            for i in it.product(*aip1):
                c += 1
                #print("c", c)
                perm1 = np.concatenate(i, axis=-1)
                #print(perm1)
                funrd[0] = perm0
                funrd[1] = perm1
                #print("funrd", funrd)
                d2 = [urd, funrd]
                d = {}
                for k in urd.keys():
                    if k in funrd.keys():
                        d[k] = np.hstack([dd[k] for dd in d2])
                    else:
                        d[k] = urd[k]
                poctpd11 = permuted_order_cauchy_to_python(d[1], 1)
                poctpd00 = permuted_order_cauchy_to_python(d[0], 0)
                p_a = a[poctpd11][:, poctpd00]
                if (p_a == b).all():
                    #print("SUCCESS!!!")
                    successful_permutation_counter = c
                    #print("successful_permutation_counter:", successful_permutation_counter)
                    rows_permutation = permuted_order_cauchy_to_python(d[1], 1)
                    cols_permutation = permuted_order_cauchy_to_python(d[0], 0)
        #print("Total number of performed brute force permutations:", c)
    else:
        print("Restart the matrices, the number of permutations is too large:", brute_force_permutations)
        
    if latex_prints:
        print("")
        print("Finally, the required permutations in python indexing style are $", list(rows_permutation+1), "$ for rows and $", list(cols_permutation+1), "$ for columns.")
    
    return rows_permutation, cols_permutation

In [31]:
m_p = 200000 # less than 5 minutes in my laptop, try 50000 for fast results
r_p, c_p = final_brute_force_latex(urd, unrd, m_p, latex_prints=True)


Now it comes the final part of the process. Currently, the ready permutations are
\begin{equation}
\left(\begin{array}{rrrr}
1 & 6 & 3 & 4 \\
2 & 1 & 4 & 3 
\end{array}\right)
,\
\left(\begin{array}{rrrrr}
6 & 5 & 3 & 2 & 8 \\
8 & 3 & 9 & 4 & 7 
\end{array}\right)
\end{equation}
The non ready permutations for columns are
\begin{equation}
\left\{\left(\begin{array}{rr}
2 & 5 \\
5 & 6 
\end{array}\right)\right\}
\end{equation}
and the non ready permutations for rows are
\begin{equation}
\left\{\left(\begin{array}{rr}
1 & 4 \\
2 & 6 
\end{array}\right)\right\}
,\
\left\{\left(\begin{array}{rr}
7 & 9 \\
1 & 5 
\end{array}\right)\right\}
\end{equation}
For the row and column indexes where we still have a degeneracy, a brute force approach is implemented, where every row in the Cauchy array contributes with the factorial of its number of columns to the total, and all the possible permutations are tested. Looking at the dimensions of the nonready Cauchy arrays $ (2, 2, 2) $ we find a total o

# Finally, the required permutations in python indexing style are:

In [32]:
r_p

array([1, 3, 8, 5, 2, 7, 0, 6, 4])

In [33]:
c_p

array([5, 0, 3, 2, 4, 1])

Python to LaTex

In [34]:
import string
import pandas as pd

In [35]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [36]:
%pprint

Pretty printing has been turned OFF


In [37]:
full_networks = np.array([[[1,1,1],
                           [1,1,0],
                           [1,0,0],
                           [1,0,0]],
                          [[1,1,1],
                           [1,1,0],
                           [1,0,0],
                           [0,1,0]],
                          [[1,1,1],
                           [1,1,0],
                           [1,0,0],
                           [0,0,1]],
                          [[1,1,1],
                           [1,1,0],
                           [0,0,1],
                           [0,0,1]]])
full_networks.shape

(4, 4, 3)

In [38]:
networks = string.ascii_uppercase[:full_networks.shape[0]]
rows = string.ascii_lowercase[:full_networks.shape[1]]
cols = string.ascii_lowercase[-full_networks.shape[2]:]
id_list = [p[0]+p[1] for p in it.product(networks, rows+cols)]
id_list
#len(id_list)

['Aa', 'Ab', 'Ac', 'Ad', 'Ax', 'Ay', 'Az', 'Ba', 'Bb', 'Bc', 'Bd', 'Bx', 'By', 'Bz', 'Ca', 'Cb', 'Cc', 'Cd', 'Cx', 'Cy', 'Cz', 'Da', 'Db', 'Dc', 'Dd', 'Dx', 'Dy', 'Dz']

In [39]:
left_x0 = 0
left_y0 = 0
delta_x = 1
delta_y = -1
left_x_locations = [left_x0 for i in range(len(rows))] + [left_x0+delta_x for i in range(len(cols))]
left_y_locations = [left_y0+delta_y*i for i in range(len(rows))] + [left_y0+delta_y*i-1/2 for i in range(len(cols))]
Delta_X = 3
x_locations = [round(j+k*Delta_X, 5) for k in [0, 1.1, 2.3, 3.5] for j in left_x_locations]#[j+k*Delta_X for k in range(len(networks)) for j in left_x_locations]#uniform distance
y_locations = left_y_locations*full_networks.shape[0]
x_locations

[0, 0, 0, 0, 1, 1, 1, 3.3, 3.3, 3.3, 3.3, 4.3, 4.3, 4.3, 6.9, 6.9, 6.9, 6.9, 7.9, 7.9, 7.9, 10.5, 10.5, 10.5, 10.5, 11.5, 11.5, 11.5]

In [40]:
#full_networks.reshape((-1,full_networks.shape[2]))
np.sum(full_networks, axis=2)

array([[3, 2, 1, 1],
       [3, 2, 1, 1],
       [3, 2, 1, 1],
       [3, 2, 1, 1]])

In [41]:
#full_networks.transpose(0,2,1)#.reshape(-1,full_networks.shape[1])
np.sum(full_networks, axis=1)

array([[4, 2, 1],
       [3, 3, 1],
       [3, 2, 2],
       [2, 2, 3]])

In [42]:
labels = []
positions = []
colors_vertices = []
colors_flat = ['teal', 'blue', 'violet', 'red']
arrays_sums_list = [np.sum(full_networks, axis=2), np.sum(full_networks, axis=1)]
shapes_product = full_networks.shape[0]*(full_networks.shape[1] + full_networks.shape[2])
shapes_sum = full_networks.shape[1] + full_networks.shape[2]
for i in range(shapes_product):
    j = i-shapes_sum*(i//shapes_sum)
    #print(i, i//shapes_sum, j)
    if j < full_networks.shape[1]:
        #print("first array")
        #print(0, i//shapes_sum, j)
        labels.append(arrays_sums_list[0][i//shapes_sum, j])
        positions.append('left')
        colors_vertices.append(colors_flat[j])
    else:
        #print("second array")
        #print(1, i//shapes_sum, j-full_networks.shape[1])
        labels.append(arrays_sums_list[1][i//shapes_sum, j-full_networks.shape[1]])
        positions.append('right')
        colors_vertices.append('black')
labels

[3, 2, 1, 1, 4, 2, 1, 3, 2, 1, 1, 3, 3, 1, 3, 2, 1, 1, 3, 2, 2, 3, 2, 1, 1, 2, 2, 3]

In [43]:
positions

['left', 'left', 'left', 'left', 'right', 'right', 'right', 'left', 'left', 'left', 'left', 'right', 'right', 'right', 'left', 'left', 'left', 'left', 'right', 'right', 'right', 'left', 'left', 'left', 'left', 'right', 'right', 'right']

In [44]:
colors_vertices

['teal', 'blue', 'violet', 'red', 'black', 'black', 'black', 'teal', 'blue', 'violet', 'red', 'black', 'black', 'black', 'teal', 'blue', 'violet', 'red', 'black', 'black', 'black', 'teal', 'blue', 'violet', 'red', 'black', 'black', 'black']

In [45]:
df_vertices = pd.DataFrame({'id': id_list,
                            'x': x_locations,
                            'y': y_locations,
                            'label': labels,
                            'position': positions,
                            'fontcolor': colors_vertices
                  })
df_vertices.to_csv(r'vertices.csv', index=False)

In [46]:
nafn = np.argwhere(full_networks)#[:,[0,1,0,2]]
nafn

array([[0, 0, 0],
       [0, 0, 1],
       [0, 0, 2],
       [0, 1, 0],
       [0, 1, 1],
       [0, 2, 0],
       [0, 3, 0],
       [1, 0, 0],
       [1, 0, 1],
       [1, 0, 2],
       [1, 1, 0],
       [1, 1, 1],
       [1, 2, 0],
       [1, 3, 1],
       [2, 0, 0],
       [2, 0, 1],
       [2, 0, 2],
       [2, 1, 0],
       [2, 1, 1],
       [2, 2, 0],
       [2, 3, 2],
       [3, 0, 0],
       [3, 0, 1],
       [3, 0, 2],
       [3, 1, 0],
       [3, 1, 1],
       [3, 2, 2],
       [3, 3, 2]])

In [47]:
u_list = [networks[r[0]]+rows[r[1]] for r in nafn]
u_list

['Aa', 'Aa', 'Aa', 'Ab', 'Ab', 'Ac', 'Ad', 'Ba', 'Ba', 'Ba', 'Bb', 'Bb', 'Bc', 'Bd', 'Ca', 'Ca', 'Ca', 'Cb', 'Cb', 'Cc', 'Cd', 'Da', 'Da', 'Da', 'Db', 'Db', 'Dc', 'Dd']

In [48]:
v_list = [networks[r[0]]+cols[r[2]] for r in nafn]
v_list

['Ax', 'Ay', 'Az', 'Ax', 'Ay', 'Ax', 'Ax', 'Bx', 'By', 'Bz', 'Bx', 'By', 'Bx', 'By', 'Cx', 'Cy', 'Cz', 'Cx', 'Cy', 'Cx', 'Cz', 'Dx', 'Dy', 'Dz', 'Dx', 'Dy', 'Dz', 'Dz']

In [49]:
colors_edges = [colors_flat[i] for i in np.argwhere(full_networks)[:,1]]
colors_edges

['teal', 'teal', 'teal', 'blue', 'blue', 'violet', 'red', 'teal', 'teal', 'teal', 'blue', 'blue', 'violet', 'red', 'teal', 'teal', 'teal', 'blue', 'blue', 'violet', 'red', 'teal', 'teal', 'teal', 'blue', 'blue', 'violet', 'red']

In [50]:
df_edges = pd.DataFrame({'u': u_list,
                         'v': v_list,
                         'color': colors_edges
                        })
df_edges.to_csv(r'edges.csv', index=False)

In [51]:
def pivots(arr):
    un_r, id_r, co_r = np.unique(np.sum(arr, axis=1), return_index=True, return_counts=True)
    unique_in_sum_row_idxs = np.sort(id_r[np.where(co_r==1)[0]])
    un_c, id_c, co_c = np.unique(np.sum(arr, axis=0), return_index=True, return_counts=True)
    unique_in_sum_col_idxs = np.sort(id_c[np.where(co_c==1)[0]])
    return unique_in_sum_row_idxs, unique_in_sum_col_idxs, arr[unique_in_sum_row_idxs, :][:, unique_in_sum_col_idxs]

In [52]:
uisri, uisci, subarr = pivots(a)

In [53]:
uisri

array([7, 8])

In [54]:
uisci

array([0])

In [55]:
subarr

array([[0],
       [0]])

# New discussion: how to quantify Gramian - Bipartite degeneracies

In [80]:
import numpy as np

def printing(algorithm_networks, *nargs):
    for arr in nargs:#[n1, n2]:
        print("")
        print(arr@arr.T)
        b = arr
        print("")
    for f in algorithm_networks:#five_networks:
        print("---")
        subt = b@b.T - f@f.T
        if (subt<0).any():
            print(subt, ": NOPE (negative)")
        else:
            argw = np.argwhere(subt>0)
            for i in np.unique(argw):
                if not ([i, i] == argw).all(1).any():
                    print(subt, ": nope (zero diagonal)")
                    break
                else:
                    print(subt, ": yes")
                    break

In [81]:
five_networks = np.array([[[1,1,1],
                           [1,1,0],
                           [1,0,0],
                           [1,0,0]],
                          [[1,1,1],
                           [1,1,0],
                           [1,0,0],
                           [0,1,0]],
                          [[1,1,1],
                           [1,1,0],
                           [1,0,0],
                           [0,0,1]],
                          [[1,1,1],
                           [1,1,0],
                           [0,0,1],
                           [1,0,0]],
                          [[1,1,1],
                           [1,1,0],
                           [0,0,1],
                           [0,0,1]]])

for arr in five_networks:
    print("")
    print(arr@arr.T)


[[3 2 1 1]
 [2 2 1 1]
 [1 1 1 1]
 [1 1 1 1]]

[[3 2 1 1]
 [2 2 1 1]
 [1 1 1 0]
 [1 1 0 1]]

[[3 2 1 1]
 [2 2 1 0]
 [1 1 1 0]
 [1 0 0 1]]

[[3 2 1 1]
 [2 2 0 1]
 [1 0 1 0]
 [1 1 0 1]]

[[3 2 1 1]
 [2 2 0 0]
 [1 0 1 1]
 [1 0 1 1]]


In [82]:
n01 = np.array([[1,1,1,0,0],
                [1,1,0,0,0],
                [1,0,0,1,0],
                [1,0,0,0,1]])

n02 = np.array([[1,1,1,0],
                [1,1,0,0],
                [1,0,0,1],
                [0,1,0,1]])

###

n03 = np.array([[1,1,1,0,0,0],
                [1,1,0,1,0,0],
                [1,0,0,0,1,1],
                [0,0,1,0,0,1]])

n04 = np.array([[1,1,1,0,0,0],
                [1,1,0,1,0,0],
                [0,0,1,1,1,0],
                [0,0,1,0,0,1]])

###

n05 = np.array([[1,1,1,0,0],
                [1,1,0,1,0],
                [1,0,0,0,1],
                [1,0,0,0,1]])

n06 = np.array([[1,1,1,0],
                [1,1,0,1],
                [0,0,1,1],
                [0,0,1,1]])

###

n07 = np.array([[1,1,1,0,0],
                [1,1,0,1,0],
                [1,0,0,0,0],
                [0,1,0,0,1]])

n08 = np.array([[1,1,1,0],
                [1,1,0,1],
                [1,0,0,0],
                [0,0,1,1]])

###

n09 = np.array([[1,1,1,0,0,0,0],
                [1,1,0,1,0,0,0],
                [1,0,0,0,1,0,0],
                [1,0,0,0,0,1,1]])

n10 = np.array([[1,1,1,0,0,0],
                [1,1,0,0,1,0],
                [1,0,0,1,0,0],
                [0,1,0,1,0,1]])

n11 = np.array([[1,1,1,0,0],
                [1,1,0,1,0],
                [1,0,0,0,1],
                [0,0,1,1,1]])

In [83]:
printing(five_networks, n01, n02)


[[3 2 1 1]
 [2 2 1 1]
 [1 1 2 1]
 [1 1 1 2]]


[[3 2 1 1]
 [2 2 1 1]
 [1 1 2 1]
 [1 1 1 2]]

---
[[0 0 0 0]
 [0 0 0 0]
 [0 0 1 0]
 [0 0 0 1]] : yes
---
[[0 0 0 0]
 [0 0 0 0]
 [0 0 1 1]
 [0 0 1 1]] : yes
---
[[0 0 0 0]
 [0 0 0 1]
 [0 0 1 1]
 [0 1 1 1]] : nope (zero diagonal)
---
[[0 0 0 0]
 [0 0 1 0]
 [0 1 1 1]
 [0 0 1 1]] : nope (zero diagonal)
---
[[0 0 0 0]
 [0 0 1 1]
 [0 1 1 0]
 [0 1 0 1]] : nope (zero diagonal)


In [84]:
printing(five_networks, n03, n04)


[[3 2 1 1]
 [2 3 1 0]
 [1 1 3 1]
 [1 0 1 2]]


[[3 2 1 1]
 [2 3 1 0]
 [1 1 3 1]
 [1 0 1 2]]

---
[[ 0  0  0  0]
 [ 0  1  0 -1]
 [ 0  0  2  0]
 [ 0 -1  0  1]] : NOPE (negative)
---
[[ 0  0  0  0]
 [ 0  1  0 -1]
 [ 0  0  2  1]
 [ 0 -1  1  1]] : NOPE (negative)
---
[[0 0 0 0]
 [0 1 0 0]
 [0 0 2 1]
 [0 0 1 1]] : yes
---
[[ 0  0  0  0]
 [ 0  1  1 -1]
 [ 0  1  2  1]
 [ 0 -1  1  1]] : NOPE (negative)
---
[[0 0 0 0]
 [0 1 1 0]
 [0 1 2 0]
 [0 0 0 1]] : yes


In [85]:
printing(five_networks, n05, n06)


[[3 2 1 1]
 [2 3 1 1]
 [1 1 2 2]
 [1 1 2 2]]


[[3 2 1 1]
 [2 3 1 1]
 [1 1 2 2]
 [1 1 2 2]]

---
[[0 0 0 0]
 [0 1 0 0]
 [0 0 1 1]
 [0 0 1 1]] : yes
---
[[0 0 0 0]
 [0 1 0 0]
 [0 0 1 2]
 [0 0 2 1]] : yes
---
[[0 0 0 0]
 [0 1 0 1]
 [0 0 1 2]
 [0 1 2 1]] : yes
---
[[0 0 0 0]
 [0 1 1 0]
 [0 1 1 2]
 [0 0 2 1]] : yes
---
[[0 0 0 0]
 [0 1 1 1]
 [0 1 1 1]
 [0 1 1 1]] : yes


In [86]:
printing(five_networks, n07, n08)


[[3 2 1 1]
 [2 3 1 1]
 [1 1 1 0]
 [1 1 0 2]]


[[3 2 1 1]
 [2 3 1 1]
 [1 1 1 0]
 [1 1 0 2]]

---
[[ 0  0  0  0]
 [ 0  1  0  0]
 [ 0  0  0 -1]
 [ 0  0 -1  1]] : NOPE (negative)
---
[[0 0 0 0]
 [0 1 0 0]
 [0 0 0 0]
 [0 0 0 1]] : yes
---
[[0 0 0 0]
 [0 1 0 1]
 [0 0 0 0]
 [0 1 0 1]] : yes
---
[[0 0 0 0]
 [0 1 1 0]
 [0 1 0 0]
 [0 0 0 1]] : yes
---
[[ 0  0  0  0]
 [ 0  1  1  1]
 [ 0  1  0 -1]
 [ 0  1 -1  1]] : NOPE (negative)


In [87]:
printing(five_networks, n09, n10, n11)


[[3 2 1 1]
 [2 3 1 1]
 [1 1 2 1]
 [1 1 1 3]]


[[3 2 1 1]
 [2 3 1 1]
 [1 1 2 1]
 [1 1 1 3]]


[[3 2 1 1]
 [2 3 1 1]
 [1 1 2 1]
 [1 1 1 3]]

---
[[0 0 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 2]] : yes
---
[[0 0 0 0]
 [0 1 0 0]
 [0 0 1 1]
 [0 0 1 2]] : yes
---
[[0 0 0 0]
 [0 1 0 1]
 [0 0 1 1]
 [0 1 1 2]] : yes
---
[[0 0 0 0]
 [0 1 1 0]
 [0 1 1 1]
 [0 0 1 2]] : yes
---
[[0 0 0 0]
 [0 1 1 1]
 [0 1 1 0]
 [0 1 0 2]] : yes
